In [6]:
import boto3

In [37]:
!pip3 install langchain-aws langchain-community langchain --quiet

## Llama3 70 Setting

In [30]:
from botocore.config import Config
from langchain_aws import ChatBedrock
bedrock_region = 'us-east-1'
modelId = "meta.llama3-70b-instruct-v1:0"
boto3_bedrock = boto3.client(
    service_name='bedrock-runtime',
    region_name=bedrock_region,
    config=Config(
        retries = {
            'max_attempts': 30
        }            
    )
)

parameters = {
    "max_gen_len": 1024,  
    "top_p": 0.9, 
    "temperature": 0.1,
}    
chat = ChatBedrock(   
    model_id=modelId,
    client=boto3_bedrock, 
    model_kwargs=parameters,
)

In [31]:
from langchain.memory import ConversationBufferWindowMemory
memory_chain = ConversationBufferWindowMemory(memory_key="chat_history", output_key='answer', return_messages=True, k=10)

## General Conversation

In [32]:
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate

def general_conversation(chat, query):
    system = (
"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n
다음은 Human과 AI의 친근한 대화입니다. AI는 상황에 맞는 구체적인 세부 정보를 충분히 제공합니다. 
AI의 이름은 서연이고, Emoji 없이 가능한 한국어로 답변하세요. 또한, 한자는 한국어로 변환합니다.<|eot_id|>"""
    )
    human = """<|start_header_id|>user<|end_header_id|>\n\n{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
    
    prompt = ChatPromptTemplate.from_messages([("system", system), MessagesPlaceholder(variable_name="history"), ("human", human)])
    print('prompt: ', prompt)
    
    chain = prompt | chat
        
    history = memory_chain.load_memory_variables({})["chat_history"]
    print('memory_chain: ', history)
                
    try: 
        stream = chain.invoke(
            {
                "history": history,
                "input": query,
            }
        )
        
        print('stream: ', stream)        
        usage = stream.response_metadata['usage']
        print('prompt_tokens: ', usage['prompt_tokens'])
        print('completion_tokens: ', usage['completion_tokens'])
        print('total_tokens: ', usage['total_tokens'])
        msg = stream.content
        
    except Exception:
        err_msg = traceback.format_exc()
        print('error message: ', err_msg)        
            
        raise Exception ("Not able to request to LLM")
    
    return msg

## 채팅 이력의 활용

In [44]:
text = "여행하고 싶어"
msg = general_conversation(chat, text)    
msg

prompt:  input_variables=['history', 'input'] input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n다음은 Human과 AI의 친근한 대화입니다. AI는 상황에 맞는 구체적인 세부 정보를 충분히 제공합니다. \nAI의 이름은 서연이고, Emoji 없이 가능한 한국어로 답변하세요. 또한, 한자는 한국어로 변환합니다.<|eot_id|>')), MessagesPlaceholder(variable_name='history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='<|start_header_id|>user<|end_header_id|>\n\n{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>'))]
memory_chain:  [HumanMessage(content='안녕'), AIMessage(content='안녕하세요. 저는 서연입니다. 오늘은 좋은 날씨네요. 요즘 어떻게 지내고 계세요?')

'여행을 계획 중이신가요? 그럼 저는 추천해 드리겠습니다. 최근에 핫한 여행지는 캄보디아의 시엠립입니다. 앙코르와트를 비롯한 문화유산과 함께 트레이딩 마켓에서 쇼핑을 즐길 수 있습니다. 또는 태국 방콕의 음식문화를 경험해 보세요.astreet food부터 고급 레스토랑까지 다양한 음식을 즐길 수 있습니다. 어디로 가시겠어요?'

In [45]:
memory_chain.chat_memory.add_user_message(text)
memory_chain.chat_memory.add_ai_message(msg)

In [46]:
text = "제주도"
msg = general_conversation(chat, text)    
msg

prompt:  input_variables=['history', 'input'] input_types={'history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\n다음은 Human과 AI의 친근한 대화입니다. AI는 상황에 맞는 구체적인 세부 정보를 충분히 제공합니다. \nAI의 이름은 서연이고, Emoji 없이 가능한 한국어로 답변하세요. 또한, 한자는 한국어로 변환합니다.<|eot_id|>')), MessagesPlaceholder(variable_name='history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='<|start_header_id|>user<|end_header_id|>\n\n{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>'))]
memory_chain:  [HumanMessage(content='안녕'), AIMessage(content='안녕하세요. 저는 서연입니다. 오늘은 좋은 날씨네요. 요즘 어떻게 지내고 계세요?')

'제주도는 좋은 선택입니다! 제주도는 대한민국의 남쪽 끝에 위치해 있는 아름다운 섬입니다. 자연경관이orgeous하고, 역사와 문화도 풍부합니다.\n\n제주도에는 많은 관광지가 있습니다. اولا, 성산일출봉은 제주도에서 가장 유명한 관광지입니다. 이 곳에서는 일출을 감상할 수 있습니다. 다음으로는 만장굴이 있습니다. 이 곳은 세계에서 가장 큰 용암동굴입니다. 또한, 오설록티뮤지엄에서는 제주도의 차 문화를 경험할 수 있습니다.\n\n제주도에는 다양한 음식도 있습니다. 블랙 포크, 黑豬,는 제주도의 특산품입니다. 또한, 제주도에서는 신선한 해산물을 즐길 수 있습니다.\n\n제주도에 가시면 무엇을 먼저 하시겠어요?'

In [47]:
memory_chain.chat_memory.add_user_message(text)
memory_chain.chat_memory.add_ai_message(msg)

In [49]:
memory_chain

ConversationBufferWindowMemory(chat_memory=InMemoryChatMessageHistory(messages=[HumanMessage(content='안녕'), AIMessage(content='안녕하세요. 저는 서연입니다. 오늘은 좋은 날씨네요. 요즘 어떻게 지내고 계세요?'), HumanMessage(content='여행하고 싶어'), AIMessage(content='여행이란 좋은 취미입니다! 요즘은 어디로 여행을 가고 싶으신가요? 국내의 경우 제주도나 부산, 경주 등이 좋고, 해외의 경우 일본, 태국, 베트남 등이 인기가 많습니다. 또는 특정 테마로 여행을 가고 싶으신가요? 예를 들어 문화탐방, 해변휴양, 산림욕 등이 있겠죠.'), HumanMessage(content='여행하고 싶어'), AIMessage(content='여행을 계획 중이신가요? 그럼 저는 추천해 드리겠습니다. 최근에 핫한 여행지는 캄보디아의 시엠립입니다. 앙코르와트를 비롯한 문화유산과 함께 트레이딩 마켓에서 쇼핑을 즐길 수 있습니다. 또는 태국 방콕의 음식문화를 경험해 보세요.astreet food부터 고급 레스토랑까지 다양한 음식을 즐길 수 있습니다. 어디로 가시겠어요?'), HumanMessage(content='제주도'), AIMessage(content='제주도는 좋은 선택입니다! 제주도는 대한민국의 남쪽 끝에 위치해 있는 아름다운 섬입니다. 자연경관이orgeous하고, 역사와 문화도 풍부합니다.\n\n제주도에는 많은 관광지가 있습니다. اولا, 성산일출봉은 제주도에서 가장 유명한 관광지입니다. 이 곳에서는 일출을 감상할 수 있습니다. 다음으로는 만장굴이 있습니다. 이 곳은 세계에서 가장 큰 용암동굴입니다. 또한, 오설록티뮤지엄에서는 제주도의 차 문화를 경험할 수 있습니다.\n\n제주도에는 다양한 음식도 있습니다. 블랙 포크, 黑豬,는 제주도의 특산품입니다. 또한, 제주도에서는 신선한 해산물을 즐길 수 있습니다.\n\n제주도에 가시면 무엇을 

In [51]:
!pip3 install bs4 --quiet

In [52]:
import requests
from bs4 import BeautifulSoup
from langchain.agents import tool

@tool
def get_product_list(keyword: str) -> list:
    """
    Search product list by keyword and then return product list
    keyword: search keyword
    return: product list
    """

    url = f"https://search.kyobobook.co.kr/search?keyword={keyword}&gbCode=TOT&target=total"
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        prod_info = soup.find_all("a", attrs={"class": "prod_info"})
        prod_list = [
            {"title": prod.text.strip(), "link": prod.get("href")} for prod in prod_info
        ]
        return prod_list[:5]
    else:
        return []

In [54]:
prod_list = get_product_list("여행")
prod_list

[{'title': '[국내도서]\n예약판매\n스마트폰과 함께하는 디지털여행',
  'link': 'https://product.kyobobook.co.kr/detail/S000213314396'},
 {'title': '[국내도서]\n예약판매\n인생은 여행',
  'link': 'https://product.kyobobook.co.kr/detail/S000213288670'},
 {'title': '[국내도서]\n예약판매\n혼자서 국내 여행(2024~2025 최신판)',
  'link': 'https://product.kyobobook.co.kr/detail/S000213304266'},
 {'title': '[국내도서]\n예약판매\n내게 말을 거는 여행의 장소',
  'link': 'https://product.kyobobook.co.kr/detail/S000213290232'},
 {'title': '[국내도서]\n여행의 이유',
  'link': 'https://product.kyobobook.co.kr/detail/S000212972861'}]

In [55]:
# langchain agent tools
tools = [get_product_list]

In [77]:
#chat = chat.bind(function=tools)
chat = chat.bind_tools(tools)

In [78]:
query = '안녕'

In [79]:
import traceback

In [82]:
system = (
"""<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n
다음은 Human과 AI의 친근한 대화입니다. AI는 상황에 맞는 구체적인 세부 정보를 충분히 제공합니다. 
AI의 이름은 서연이고, Emoji 없이 가능한 한국어로 답변하세요. 또한, 한자는 한국어로 변환합니다.<|eot_id|>"""
    )
human = """<|start_header_id|>user<|end_header_id|>\n\n{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""
    
# prompt = ChatPromptTemplate.from_messages([("system", system), MessagesPlaceholder(variable_name="agent_scratchpad"), ("human", human)])

propmt = ChatPromptTemplate.from_messages([
    ("system", system),
    ("user", human),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

chain = prompt | chat
        
history = memory_chain.load_memory_variables({})["chat_history"]
print('memory_chain: ', history)
                
try: 
    stream = chain.invoke(
        {
            # "history": history,
            "input": query,
            "agent_scratchpad": ["여행"]
        }
    )
        
    print('stream: ', stream)        
    usage = stream.response_metadata['usage']
    print('prompt_tokens: ', usage['prompt_tokens'])
    print('completion_tokens: ', usage['completion_tokens'])
    print('total_tokens: ', usage['total_tokens'])
    msg = stream.content
except Exception:
    err_msg = traceback.format_exc()
    print('error message: ', err_msg)        
            
    raise Exception ("Not able to request to LLM")

memory_chain:  [HumanMessage(content='안녕'), AIMessage(content='안녕하세요. 저는 서연입니다. 오늘은 좋은 날씨네요. 요즘 어떻게 지내고 계세요?'), HumanMessage(content='여행하고 싶어'), AIMessage(content='여행이란 좋은 취미입니다! 요즘은 어디로 여행을 가고 싶으신가요? 국내의 경우 제주도나 부산, 경주 등이 좋고, 해외의 경우 일본, 태국, 베트남 등이 인기가 많습니다. 또는 특정 테마로 여행을 가고 싶으신가요? 예를 들어 문화탐방, 해변휴양, 산림욕 등이 있겠죠.'), HumanMessage(content='여행하고 싶어'), AIMessage(content='여행을 계획 중이신가요? 그럼 저는 추천해 드리겠습니다. 최근에 핫한 여행지는 캄보디아의 시엠립입니다. 앙코르와트를 비롯한 문화유산과 함께 트레이딩 마켓에서 쇼핑을 즐길 수 있습니다. 또는 태국 방콕의 음식문화를 경험해 보세요.astreet food부터 고급 레스토랑까지 다양한 음식을 즐길 수 있습니다. 어디로 가시겠어요?'), HumanMessage(content='제주도'), AIMessage(content='제주도는 좋은 선택입니다! 제주도는 대한민국의 남쪽 끝에 위치해 있는 아름다운 섬입니다. 자연경관이orgeous하고, 역사와 문화도 풍부합니다.\n\n제주도에는 많은 관광지가 있습니다. اولا, 성산일출봉은 제주도에서 가장 유명한 관광지입니다. 이 곳에서는 일출을 감상할 수 있습니다. 다음으로는 만장굴이 있습니다. 이 곳은 세계에서 가장 큰 용암동굴입니다. 또한, 오설록티뮤지엄에서는 제주도의 차 문화를 경험할 수 있습니다.\n\n제주도에는 다양한 음식도 있습니다. 블랙 포크, 黑豬,는 제주도의 특산품입니다. 또한, 제주도에서는 신선한 해산물을 즐길 수 있습니다.\n\n제주도에 가시면 무엇을 먼저 하시겠어요?')]
stream:  content="Travel! 🗺️ I'd be happy to help y

In [83]:
msg

"Travel! 🗺️ I'd be happy to help you plan a trip or provide information about different destinations. Where are you thinking of going? Are you looking for recommendations on popular tourist spots, cultural experiences, or adventure activities? 🤔"

In [74]:
# Prompt 정의
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [76]:
agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
)

TypeError: Expected a Runnable, callable or dict.Instead got an unsupported type: <class 'list'>

In [ ]:
def run(agent_executor: AgentExecutor, keyword: str) -> dict:
    result = agent_executor.invoke(
        {
            "input": f"""
        1. `{keyword}` 키워드로 상품 리스트를 가져와줘.
        2. 1번 과정에서 가져온 상품 리스트 각각의 table of content 를 가져와줘.
        3. 마지막으로 모든 상품 리스트의 table of content 를 종합하여 내가 앞으로 집필할 베스트셀러 책의 목차를 작성해 줘.
        """
        }
    )
    return result["output"]

In [ ]:
final_answer = run(agent_executor, "Pandas")